# 第9章 控制抽象
在第7章指出Scala并没有很多内建的控制抽象，因为它提供了让你自己创建控制抽象的能力。在前一章，你学到了函数值。本章将向你展示如何应用函数值来创建新的控制抽象。在这个过程中，你还将学习到柯里化和传名参数。

## 9.1 减少代码重复
所有的函数都能被分解成每次函数调用都一样的公共的部分和每次调用不一样的非公共部分。公共部分是函数体，而非公共部分必须通过实参传入。当你把函数值当作入参的时候，这段算法的非公共部分本身又是另一个算法！每当这样的函数被调用，你都可以传入不同的函数值作为实参，被调用的函数会（在由它选择的时机）调用传入的函数值。这些高阶函数（higher-order function），即那些接收函数作为参数的函数，让你有额外的机会来进一步压缩和简化代码。

高阶函数的好处之一是可以用来创建减少代码重复的控制抽象。例如，假定你在编写一个文件浏览器，而你打算提供API给用户来查找匹配某个条件的文件。首先，你添加了一个机制用来查找文件名是以指定字符串结尾的文件。比如，这将允许用户查找所有扩展名为“.scala”的文件。可以通过在单例对象中定义一个公共的filesEnding方法的方式来提供这样的API，就像这样：

In [2]:
object FileMatcher {
    private def filesHere = (new java.io.File(".")).listFiles
    def fileEnding(query: String) = 
        for(file <- filesHere; if file.getName.endsWith(query))
            yield file
}

defined object FileMatcher

这个filesEnding方法用私有的助手方法filesHere来获取当前目录下的所有文件，然后基于文件名是否以用户给定的查询条件结尾来过滤这些文件。由于filesHere是私有的，filesEnding方法是FileMatcher（也就是你提供给用户的API）中定义的唯一一个能被访问到的方法。

目前为止一切都很完美，暂时都还没有重复的代码。不过到了后来，你决定要让人们可以基于文件名的任意部分进行搜索。当用户记不住他们到底是将文件命名成了phb-important.doc、stupid-phb-report.doc、may2003salesdoc.phb还是别的什么完全不一样的名字，他们只知道名字中某个地方出现了“phb”，这个时候这样的功能就很有用。于是回去给你的FileMatcher API添加了这个函数：

In [3]:
object FileMatcher {
    private def filesHere = (new java.io.File(".")).listFiles
    def fileEnding(query: String) = 
        for(file <- filesHere; if file.getName.endsWith(query))
            yield file
    def fileContaining(query: String) = 
        for(file <- filesHere; if file.getName.contains(query))
            yield file
}

defined object FileMatcher

这个函数跟filesEnding的运行机制没什么两样：搜索filesHere，检查文件名，如果名字匹配则返回文件。唯一的区别是这个函数用的是contains而不是endsWith。

几个月过去了，这个程序变得更成功了。终于，你对某些高级用户提出的想要基于正则表达式搜索文件的请求屈服了。这些喜欢偷懒的用户有着大量拥有上千个文件的巨大目录，他们想做到类似找出所有标题中带有“oopsla”字样的“pdf”文件。为了支持他们，编写了下面这个函数：

In [4]:
object FileMatcher {
    private def filesHere = (new java.io.File(".")).listFiles
    def fileEnding(query: String) = 
        for(file <- filesHere; if file.getName.endsWith(query))
            yield file
    def fileContaining(query: String) = 
        for(file <- filesHere; if file.getName.contains(query))
            yield file
    def fileRegex(query: String) = 
        for(file <- filesHere; if file.getName.matches(query))
            yield file
}

defined object FileMatcher

In [4]:
def fileMatching(query: String, method) = 
    for(file <- filesHere; if file.getName.method(query))
        yield file

(console):1: ':' expected but ')' found.
def fileMatching(query: String, method) = 
                                      ^
(console):2: identifier expected but ';' found.
    for(file <- filesHere; if file.getName.method(query))
                         ^
(console):2: '(' expected but identifier found.
    for(file <- filesHere; if file.getName.method(query))
                              ^
(console):3: illegal start of simple expression
        yield file
                  ^

: 

这种方式在某些动态语言中可以做到，但Scala并不允许像这样在运行时将代码黏在一起。那怎么办呢？

函数值提供了一种答案。虽然不能将方法名像值一样传来传去，但是可以通过传递某个帮你调用方法的函数值来达到同样的效果。在本例中，可以给方法添加一个matcher参数，该参数唯一的目的就是检查文件名是否满足某个查询条件：

In [5]:
object FileMatcher {
    private def filesHere = (new java.io.File(".")).listFiles
    def fileEnding(query: String) = 
        for(file <- filesHere; if file.getName.endsWith(query))
            yield file
    def fileContaining(query: String) = 
        for(file <- filesHere; if file.getName.contains(query))
            yield file
    def fileRegex(query: String) = 
        for(file <- filesHere; if file.getName.matches(query))
            yield file
    def filesMatching(query: String,
        matcher: (String, String) => Boolean) = {
        for (file <- filesHere; if matcher(file.getName, query))
            yield file
    }
}

defined object FileMatcher

在这个版本的方法中，if子句用matcher来检查文件名是否满足查询条件。这个检查具体做什么，取决于给定的matcher。现在，我们来看看matcher这个类型本身。它首先是个函数，因此在类型声明中有个=>符号。这个函数接收两个字符串类型的参数（分别是文件名和查询条件），返回一个布尔值，因此这个函数的完整类型是（String, String） => Boolean。

有了这个新的filesMatching助手方法，可以将前面三个搜索方法进行简化，调用助手方法，传入合适的函数：

In [6]:
object FileMatcher {
    private def filesHere = (new java.io.File(".")).listFiles
    def filesMatching(query: String,
        matcher: (String, String) => Boolean) = {
        for (file <- filesHere; if matcher(file.getName, query))
            yield file
    }
    def fileEnding(query: String) = 
        filesMatching(query, _.endsWith(_))
    def fileContaining(query: String) = 
        filesMatching(query, _.contains(_))
    def fileRegex(query: String) = 
        filesMatching(query, _.matches(_))
}

defined object FileMatcher